# Data Preprocsesing

In [1]:
import json

In [5]:
with open('data/trec_corpus_20220301_plain.json') as read_from_file:
    with open('data/preprocessed_corputs.csv', 'a') as write_to_file:
        
        # write header of csv file
        write_to_file.write('id, text')
        doc = read_from_file.readline()

        for i in range(5): # while doc:
            
            # parse json
            doc_parsed = json.loads(doc)
            doc_id = doc_parsed['id']
            doc_text = doc_parsed['title'] + ' ' + doc_parsed['plain']
            
            print(doc_parsed)
            print('\n\n############\n\n')

            # do preprocessing
            preprocessed_text = 'test test test test test'

            # write preprocessed document to file (append)
            write_to_file.write(f'{doc_id}, {preprocessed_text}\n')

            # read next line
            doc = read_from_file.readline()

{'id': 12148915, 'title': 'Keith Osik', 'url': 'https://en.wikipedia.org/wiki/Keith_Osik', 'plain': 'Keith Richard Osik (born October 22, 1968), is a former Major League Baseball catcher who played in the major leagues from -. He played for the Milwaukee Brewers, Pittsburgh Pirates, Baltimore Orioles, and Washington Nationals. He was drafted in the 24th round of the  MLB Draft. His brother was also a professional baseball player, but only played in the minors. He was born in Port Washington, New York, but now lives in Shoreham, New York.\n\nOsik is currently the head baseball coach at Farmingdale State College, a Division III institution located on Long Island in New York. He was inducted into the Suffolk Sports Hall of Fame on Long Island in the Baseball Category with the Class of 2008.\n\nExternal links\n\n1968 births\nLiving people\nMajor League Baseball catchers\nBaseball players from New York (state)\nPeople from Port Washington, New York\nMilwaukee Brewers players\nPittsburgh Pir